<a href="https://colab.research.google.com/github/pigsoft/Test/blob/master/PyTorch_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

import torch

dtype = torch.double
device = torch.device("cpu")
# device = torch.device("cuda")

In [0]:
class all_activation_function:
    def __init__(self):
        pass
        
    def Relu(self, x) :
        return x*(x>0)
    
    def d_Relu(self, x) :
        return 1.0*(x>0)
    
    def PRelu(self, x, alpha) :
        return x*(x>=0) + alpha*x*(x<0)
    
    def d_PRelu(self, x, alpha) :
        return 1.0*(x>=0) + alpha*(x<0)
    
    def Tanh(self, x) :
        return torch.tanh(x)
    
    def d_Tanh(self, x) :
        return 1 - torch.tanh(x)**2
    
    def sigmoid(self, x) :
        return 1.0/(1 + torch.exp(-x))
    
    def d_sigmoid(self, x) :
        tmp = torch.exp(-x)
        return tmp/((1-tmp)**2)
    
    def Elu(self, x, alpha) :
        return alpha*(torch.exp(x)-1)*(x<=0) + x*(x>0)
    
    def d_Elu(self, x, alpha) :
        return alpha * torch.exp(x) * (x<=0) + 1.0 * (x>0)
    
    def softmax(self, x):
        return torch.exp(x) / torch.reshape(torch.sum(torch.exp(x), axis=1), (-1, 1))
    
    def log_softmax(self, x):
        return torch.log(self.softmax(x))
    
    def CrossEntropy(self, x, y):
        log_softmax = -self.log_softmax(x)
        
        expected_loss = torch.zeros(1, dtype=dtype)
        for row, col in enumerate(y):
            expected_loss += log_softmax[row, col]
        expected_loss /= log_softmax.shape[0]

        return expected_loss

In [0]:
class neural_network(all_activation_function):

    def __init__(self, X, y):
        self.X_train = torch.from_numpy(X).to(device=device)
        self.X = torch.from_numpy(X).to(device=device)
        self.y = torch.from_numpy(y)
        self.W = []
        self.b = []
        self.n_node = []
        self.activation_func = []
        self.loss_func = None
        self.list_loss = []
        self.loss_ = None

        self.dict_activation_func = {"sigmoid": self.sigmoid,
                                     "softmax": self.softmax,
                                     "softmax_cross_entropy": self.CrossEntropy
                                     }

    def linear(self, X, W, b):
        Z = X @ W + b
        return Z

    def layer(self, n_node, activation_func):
        self.n_node.append(n_node)
        self.activation_func.append(activation_func)

    def loss(self, n_node, loss_func):
        self.n_node.append(n_node)
        self.loss_func = loss_func
    
    def forward_train(self, epoch):
        # Set X_train to original value for forward 
        self.X = self.X_train

        # i = 0 (first layer) to len(self.n_node) - 1 
        for i in range(len(self.n_node) - 1):
            n_prev_node = self.X.shape[1]
            n_curr_node = self.n_node[i]

            Z = self.linear(self.X, self.W[i], self.b[i])
            self.X = self.dict_activation_func[self.activation_func[i]](Z)
        
        # Last layer (loss function)        
        Z = self.linear(self.X, self.W[-1], self.b[-1])
        self.loss = self.dict_activation_func[self.loss_func](Z, self.y)
        self.list_loss.append(self.loss.data)
        
        if epoch % 100 == 0:
            print('epoch: {}, \tloss: {}'.format(epoch, self.loss.data[0]))
    
    def forward_test(self):
        # Set X_train to original value for forward 
        self.X = self.X_test

        # i = 0 (first layer) to len(self.n_node) - 1 
        for i in range(len(self.n_node) - 1):
            n_prev_node = self.X.shape[1]
            n_curr_node = self.n_node[i]
            
            Z = self.linear(self.X, self.W[i], self.b[i])
            self.X = self.dict_activation_func[self.activation_func[i]](Z)
        
        # Last layer (loss function)        
        Z = self.linear(self.X, self.W[-1], self.b[-1])
        predicted = torch.argmax(Z, axis=1)
        return predicted

    def backward(self, lr):
        # Backpropagation
        self.loss.backward()
        for i in range(len(self.W)):
            with torch.no_grad():
                self.W[i] -= self.W[i].grad * lr
                self.b[i] -= self.b[i].grad * lr

                self.W[i].grad.zero_()
                self.b[i].grad.zero_()

    def fit(self, n_epoch=2000, lr=1e-6):
        # Create weights and biases
        n_prev_node = self.X_train.shape[1]
        for i in range(len(self.n_node)):
            n_curr_node = self.n_node[i]
            
            torch.manual_seed(0)
            W = torch.randn(n_prev_node, n_curr_node, device=device, dtype=dtype, requires_grad=True)
            b = torch.randn(n_curr_node, device=device, dtype=dtype, requires_grad=True)
            self.W.append(W)
            self.b.append(b)

            n_prev_node = self.n_node[i]
        
        for epoch in range(n_epoch):
            self.forward_train(epoch)
            self.backward(lr)

    def predict(self, X_test):
        self.X_test = torch.from_numpy(X_test)
        predicted = self.forward_test()
        return predicted
        

In [0]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

digits = load_digits()
X = np.array(digits.data)
y = np.array(digits.target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(type(X_train[0][0]))
print(type(y_train[0]))

(1203, 64)
(594, 64)
(1203,)
(594,)
<class 'numpy.float64'>
<class 'numpy.int64'>


In [0]:
nn = neural_network(X_train, y_train)
nn.layer(10, 'sigmoid')
nn.layer(10, 'sigmoid')
nn.layer(10, 'sigmoid')
nn.loss(10, 'softmax_cross_entropy')
nn.fit(n_epoch=1001, lr=1.5)

epoch: 0, 	loss: 4.2598983684014256
epoch: 100, 	loss: 1.100952897324285
epoch: 200, 	loss: 0.6706005001694546
epoch: 300, 	loss: 0.4698918088558004
epoch: 400, 	loss: 0.3514013198696748
epoch: 500, 	loss: 0.3053528031275973
epoch: 600, 	loss: 0.24273944407598716
epoch: 700, 	loss: 0.2328103278297493
epoch: 800, 	loss: 0.1765971861140311
epoch: 900, 	loss: 0.16669038727906157
epoch: 1000, 	loss: 0.15675706870729442


In [0]:
from sklearn.metrics import accuracy_score

y_predicted = nn.predict(X_test)
acc = accuracy_score(y_test, y_predicted)
print('Accuracy:', acc * 100.0)

Accuracy: 88.38383838383838
